# Wagon load balancing

Three railway wagons with a carrying capacity of 100 quintals (1 quintal = 100 kg) have been reserved to transport sixteen boxes. The weight of the boxes in quintals is given. How shall the boxes be assigned to the wagons in order to keep to the limits on the maximum carrying capacity and to minimize the heaviest wagon load?.

In this problem we wish to minimize the maximum load of the wagons. Such an objective is sometimes referred to as $\textbf{minimax}$ objective. These kind of problems, like $\textbf{maximin}$ problems are also called $\textbf{bottleneck problems}$. The procedure to represent a bottleneck criterion in a linear model is:

- We define a non-negative variable, $W$ in this case, to represent the maximum weight over all the wagon loads.
- Constraints are established to set $W$ as the upper bound on every wagon load.
- The objective function consists of minimizing $W$.

### Mathematical Programming Model
#### Sets:
- $B$: Boxes to load.
- $N$: Wagons.

#### Inputs:
- $w_{b}$: Weight of box $b$.

#### Decision Variables:
- $x_{bn}$: 1 if box $b$ is assigned to wagon $n$, 0 otherwise.
- $W$: Maximum weight over all the wagon loads.

#### Objective Function: Minimize the weight of the heaviest wagon load.
$$\text{min} \quad W$$

#### Constraints:
1. All boxes must be assigned.
$$\sum_{n \in N} x_{bn} = 1 \qquad \forall b \in B$$

2. The total weight of a wagon must be less than or equal to the maximum weight over all the wagon loads.
$$\sum_{b \in B} x_{bn} w_{b} \leq W \qquad \forall n \in N$$

3. Box allocation is binary.
$$x_{bn} \in \{0,1\} \qquad \forall b \in B, n \in N$$

4. Maximum weight is non-negative.
$$W \geq 0$$

In [1]:
import pyomo.environ as pyo
import pandas as pd
import numpy as np

In [4]:
pd.read_excel("./data/wagon_load_balancing.xlsx", sheet_name=None)

{'Boxes':     Box  Weight
 0     1      34
 1     2       6
 2     3       8
 3     4      17
 4     5      16
 5     6       5
 6     7      13
 7     8      21
 8     9      25
 9    10      31
 10   11      14
 11   12      13
 12   13      33
 13   14       9
 14   15      25
 15   16      25,
 'Wagons':    Wagon  Capacity
 0      1       100
 1      2       100
 2      3       100}

In [5]:
boxes = pd.read_excel("./data/4_wagon_load_balancing.xlsx", sheet_name="Boxes", index_col=0)
boxes

,Weight
Box,
1,34
2,6
3,8
4,17
5,16
6,5
7,13
8,21
9,25


In [7]:
wagons = pd.read_excel("./data/4_wagon_load_balancing.xlsx", sheet_name="Wagons", index_col=0)
wagons

,Capacity
Wagon,
1,100
2,100
3,100


In [8]:
B = boxes.index
N = wagons.index
wb = boxes["Weight"]

In [14]:
model = pyo.ConcreteModel("Wagon load balancing")

model.x = pyo.Var(B, N, domain=pyo.Binary, doc="1 if box b is assigned to wagon n, 0 otherwise")
model.W = pyo.Var(domain=pyo.NonNegativeReals, doc="Weight of the heaviest wagon load")

model.objective = pyo.Objective(expr=model.W, sense=pyo.minimize, doc="Minimize the maximum weight of the heaviest wagon load")

model.constraint1 = pyo.ConstraintList(doc="All boxes must be allocated")
for b in B:
    model.constraint1.add(expr=sum(model.x[b,n] for n in N) == 1)

model.constraint2 = pyo.ConstraintList(doc="The total weight of a wagon must be less than or equal to the maximum weight over all the wagon loads")
for n in N:
    model.constraint2.add(expr=sum(wb[b]*model.x[b,n] for b in B) <= model.W)

solver = pyo.SolverFactory("glpk")
results = solver.solve(model)
results

{'Problem': [{'Name': 'unknown', 'Lower bound': 99.0, 'Upper bound': 99.0, 'Number of objectives': 1, 'Number of constraints': 19, 'Number of variables': 49, 'Number of nonzeros': 99, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': '136057', 'Number of created subproblems': '136057'}}, 'Error rc': 0, 'Time': 10.427249193191528}], 'Solution': [OrderedDict({'number of solutions': 0, 'number of solutions displayed': 0})]}

In [20]:
for n in N:
    for b in B:
        if pyo.value(model.x[b,n]) != 0:
            print(f"Box: {b} is assigend to wagon: {n}")

Box: 1 is assigend to wagon: 1
Box: 2 is assigend to wagon: 1
Box: 13 is assigend to wagon: 1
Box: 15 is assigend to wagon: 1
Box: 5 is assigend to wagon: 2
Box: 6 is assigend to wagon: 2
Box: 7 is assigend to wagon: 2
Box: 9 is assigend to wagon: 2
Box: 10 is assigend to wagon: 2
Box: 14 is assigend to wagon: 2
Box: 3 is assigend to wagon: 3
Box: 4 is assigend to wagon: 3
Box: 8 is assigend to wagon: 3
Box: 11 is assigend to wagon: 3
Box: 12 is assigend to wagon: 3
Box: 16 is assigend to wagon: 3


In [23]:
print(f"The minimum maximum weight of the heaviest wagon load is {pyo.value(model.objective)}")

The minimum maximum weight of the heaviest wagon load is 99.0
